En el siguiente notebook vamos a trabajar con PySpark. En él, vamos a usar las transformaciones y las acciones para analizar un pequeño conjunto de datos.

El primer paso que debemos dar es conectar Google Colab con nuestro Google Drive. Para ello, lanzaremos el siguiente trozo de código.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


Una vez hemos conectado Google Drive, pasamos a instalar Apache Spark en nuestro notebook. En concreto, vamos a trabajar con la versión 3.5.0.

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [5]:
!ls


drive  sample_data  spark-3.5.0-bin-hadoop3.tgz


In [6]:
!tar xf spark-3.5.0-bin-hadoop3.tgz

Además de la instalación habitual, hay un paso más que debemos dar, y es la instalación de la librería *findspark*. Esta librería nos permitirá encontrar la instalación de Apache Spark en nuestro sistema.

Establecemos también un par de variables de entorno.

In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [9]:
import findspark
findspark.init()

In [10]:
findspark.find()

'/content/spark-3.5.0-bin-hadoop3'

Una vez tenemos instalado nuestro framework, es momento de crear nuestra variable SparkSession, de la cual parten la mayoría de funcionalidades de SparkSQL

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [12]:
spark

### Lectura de datos

Ahora sí, podemos comenzar a trabajar con SparkSQL. Comenzaremos leyendo nuestro conjunto de datos, el cual contiene información de valoraciones de restaurantes.

In [13]:
!pwd

/content


In [14]:
import pandas as pd

data_path = '/content/drive/MyDrive/TokioSchool/'

data = spark.read.options(inferSchema='True',delimiter=',', header=True).csv (data_path + 'CSV_stocks_2021.csv')


In [15]:
data

DataFrame[ticker: string, open: double, high: double, low: double, close: double, volume: int, dividends: double, stock splits: int, date: date, ccy: string]

### Análisis estadísico

In [16]:
data.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- dividends: double (nullable = true)
 |-- stock splits: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- ccy: string (nullable = true)



Obtención de datos

*   a) Realizaremos un muestreo conforme a las siguientes especificaciones:
• HON: 3%
• WMT: 5%
• BA: 1%




In [20]:
data.sampleBy('ticker', fractions={'HON': 0.03,
                                   'WTM': 0.05,
                                   'BA': 0.01}).show()

+------+------------------+------------------+------------------+------------------+--------+---------+------------+----------+---+
|ticker|              open|              high|               low|             close|  volume|dividends|stock splits|      date|ccy|
+------+------------------+------------------+------------------+------------------+--------+---------+------------+----------+---+
|   HON|207.45501408335315|209.42141232111007|206.14735745495346|209.12644958496094| 1406400|      0.0|           0|2020-12-31|USD|
|   HON|204.76875508579627|205.45015278172468|198.01389317021344| 200.4136505126953| 2798800|      0.0|           0|2021-03-04|USD|
|   HON| 209.1534615359291|212.08648979867974|208.57080994579175|209.85462951660156| 2959700|      0.0|           0|2021-03-11|USD|
|   HON|225.86941915430066|226.44463178520627|222.11072065973417| 223.3206329345703| 1998600|      0.0|           0|2021-06-11|USD|
|   HON|217.26109074085943|219.52132727211043|215.83725126864638|217.3805847